In [7]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk

In [2]:
!pip install transformers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 109.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.1 MB/s eta 0:00:00


Pre trained Summarization model

In [3]:
import pandas as pd
file_path = 'dataset_class.csv'
encoding = 'utf-8'
df = pd.read_csv('/content/dataset_final.csv', encoding=encoding)
#df = pd.read_csv('/content/all-data.csv', sep='\t', header=None, names=['labels', 'text'],encoding=encoding)


In [4]:
y_count = df["labels"].value_counts()
y_count

0    1126
2    1108
1    1104
Name: labels, dtype: int64

In [5]:
df.head()

,labels,Text
0,2,Technopolis plans to develop in stages an area...
1,2,At the request of Finnish media company Alma M...
2,2,"In Sweden , Gallerix accumulated SEK denominat..."
3,2,The company supports its global customers in d...
4,2,"When this investment is in place , Atria plans..."


In [6]:
df.tail()

,labels,Text
3333,2,Emergency Response Infrastructure: Development...
3334,2,Infrastructure Resilience Planning: Planning f...
3335,2,Land Use Planning: Integration of infrastructu...
3336,2,Accessibility Upgrades: Upgrades to make publi...
3337,2,Infrastructure Data Collection: Centralized da...


In [53]:
!pip install sentencepiece


In [1]:
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification

# Load the pre-trained XLM-RoBERTa model and tokenizer
model_name = "xlm-roberta-base"
tokenizer = XLMRobertaTokenizer.from_pretrained(model_name)
num_labels = 3  # Modify this according to your specific task
model = XLMRobertaForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# from transformers import BertTokenizer, BertForSequenceClassification, TrainingArguments, Trainer
# import torch

# # Load the pre-trained mBERT model and tokenizer
# model_name = "bert-base-multilingual-cased"
# tokenizer = BertTokenizer.from_pretrained(model_name)
# num_labels = 3  # Modify this according to your specific task
# model = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
# label_map = {"positive": 0, "negative": 1, "neutral": 2}
# df['labels'] = df['labels'].map(label_map)

In [9]:
df.head()

,labels,Text
0,2,Technopolis plans to develop in stages an area...
1,2,At the request of Finnish media company Alma M...
2,2,"In Sweden , Gallerix accumulated SEK denominat..."
3,2,The company supports its global customers in d...
4,2,"When this investment is in place , Atria plans..."


In [4]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df,test_size=0.1, random_state=42)


In [5]:
train_encodings = tokenizer(list(train_df['Text']), truncation=True, padding=True, return_tensors='pt', max_length=128)
test_encodings = tokenizer(list(test_df['Text']), truncation=True, padding=True, return_tensors='pt', max_length=128)

In [8]:
import torch
train_labels = torch.tensor(list(train_df['labels']))
test_labels = torch.tensor(list(test_df['labels']))


In [9]:
from torch.utils.data import DataLoader, TensorDataset
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], train_labels)
test_dataset = TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], test_labels)

In [10]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

In [11]:
import torch
import torch.optim as optim
import torch.nn as nn

# Set device to GPU if available, otherwise use CPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768,

In [12]:
from transformers import AdamW
import torch
optimizer = AdamW(model.parameters(), lr=2e-5)  # Adjust learning rate as needed
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)



/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


XLMRobertaForSequenceClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768,

In [13]:
import time

num_epochs = 5


  # Adjust as needed
total_batches = len(train_loader)
for epoch in range(num_epochs):
    start_time = time.time()  # Record the start time for each epoch
    model.train()

    for batch_idx, batch in enumerate(train_loader):
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        # Calculate training progress as a percentage
        progress = (batch_idx + 1) / total_batches * 100

        print(f"Epoch {epoch+1}/{num_epochs}, Batch {batch_idx+1}/{total_batches}, Progress: {progress:.2f}%", end="\r")

    end_time = time.time()  # Record the end time for each epoch
    epoch_time = end_time - start_time  # Calculate the time taken for the epoch
    print(f"\nEpoch {epoch+1}/{num_epochs} completed in {epoch_time:.2f} seconds")


Epoch 1/5, Batch 94/94, Progress: 100.00%
Epoch 1/5 completed in 57.88 seconds
Epoch 2/5, Batch 94/94, Progress: 100.00%
Epoch 2/5 completed in 59.79 seconds
Epoch 3/5, Batch 94/94, Progress: 100.00%
Epoch 3/5 completed in 61.51 seconds
Epoch 4/5, Batch 94/94, Progress: 100.00%
Epoch 4/5 completed in 62.16 seconds
Epoch 5/5, Batch 94/94, Progress: 100.00%
Epoch 5/5 completed in 63.61 seconds


In [14]:
model.eval()
correct_predictions = 0
total_predictions = 0
with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        predictions = torch.argmax(outputs.logits, dim=1)

        correct_predictions += (predictions == labels).sum().item()
        total_predictions += len(labels)

accuracy = correct_predictions / total_predictions
print(f"Accuracy on Test Data: {accuracy}")

Accuracy on Test Data: 0.8353293413173652


In [15]:
from sklearn.metrics import accuracy_score, classification_report

all_preds = []
all_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1).cpu().numpy()
        all_preds.extend(preds)
        all_labels.extend(labels.cpu().numpy())

accuracy = accuracy_score(all_labels, all_preds)
report = classification_report(all_labels, all_preds)

print(f"Accuracy: {accuracy}")
print(report)

Accuracy: 0.8353293413173652
              precision    recall  f1-score   support

           0       0.68      0.90      0.78        94
           1       0.92      0.94      0.93        99
           2       0.94      0.72      0.81       141

    accuracy                           0.84       334
   macro avg       0.85      0.85      0.84       334
weighted avg       0.86      0.84      0.84       334



In [16]:
label_map = {"positive": 0, "negative": 1, "neutral": 2}


In [17]:
inv_map = {v: k for k, v in label_map.items()}

In [18]:
def classify_text(text):
    # Tokenize and encode the input text
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    inputs.to(device)

    # Perform inference
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits

    # Get the predicted class (assuming a single text input)
    predicted_class = torch.argmax(logits, dim=1).item()

    return inv_map[predicted_class]

In [19]:
classify_text("India Reports Surge in COVID-19 Cases Despite Vaccination Efforts:India has witnessed a concerning increase in COVID-19 infections, even as vaccination campaigns continue.")

'negative'

In [20]:
classify_text("தடுப்பூசி முயற்சிகள் இருந்தபோதிலும், கோவிட்-19 வழக்குகள் அதிகரிப்பதாக இந்தியா தெரிவிக்கிறது:தடுப்பூசி பிரச்சாரங்கள் தொடர்ந்தாலும், கோவிட்-19 நோய்த்தொற்றுகளின் எண்ணிக்கை அதிகரிப்பதை இந்தியா கண்டுள்ளது. புதிய மாறுபாடுகள் மற்றும் தளர்வான பாதுகாப்பு நடவடிக்கைகளே இந்த உயர்வுக்கு காரணம் என நிபுணர்கள் கூறுகின்றனர். சுகாதார உள்கட்டமைப்பை மேம்படுத்தவும், வைரஸ் பரவுவதைத் தடுக்க தடுப்பூசி முயற்சிகளை துரிதப்படுத்தவும் அதிகாரிகள் பணியாற்றி வருகின்றனர்.")

'positive'

In [21]:
classify_text("Economic recession hits India hard: Center faces criticism for rising unemployment")

'negative'

In [22]:
classify_text("பொருளாதார மந்தநிலை இந்தியாவை கடுமையாக தாக்குகிறது: அதிகரித்து வரும் வேலையில்லா திண்டாட்டத்திற்கு மத்திய அரசு விமர்சனத்தை எதிர்கொள்கிறது")

'negative'

In [23]:
classify_text("விவசாயிகள் போராட்டம் தீவிரமடைந்து, சீர்திருத்தங்களை மறுமதிப்பீடு செய்ய மத்திய அரசுக்கு அழுத்தம் கொடுக்கிறது.")

'negative'

In [24]:
classify_text("மத்திய அரசு உயர் பதவிகளில் ஊழல் குற்றச்சாட்டுகளை சுமத்தி வருகிறது.")

'negative'

In [25]:
classify_text("முக்கிய உள்கட்டமைப்பு திட்டங்கள் ஸ்டால், மத்திய அரசின் செயல்திறன் குறித்த கவலைகளை எழுப்புகிறது.")

'negative'

In [26]:
classify_text("விண்வெளி சாதனைகள்: இந்தியாவின் விண்வெளி நிறுவனமான இஸ்ரோ, விண்வெளி ஆராய்ச்சியில் குறிப்பிடத்தக்க முன்னேற்றங்களைச் செய்துள்ளது, இதில் வெற்றிகரமான செயற்கைக்கோள் ஏவுதல்கள் மற்றும் சந்திரயான் மற்றும் மங்கள்யான் போன்ற பணிகள், சர்வதேச அங்கீகாரத்தைப் பெற்றன.")


'positive'

In [27]:
classify_text("பாராளுமன்ற அறிவிப்பு: இந்திய மத்திய அரசு அறிவியல் மற்றும் பொருளாதார விவகாரங்களின் நிலை குறித்த அறிவிப்பை வெளியிட்டுள்ளது.")

'neutral'

In [28]:
classify_text("Parliamentary Notification: The Central Government of India has issued a notification on the status of scientific and economic affairs.")

'neutral'

In [29]:
classify_text("Space Achievements: India's space agency, ISRO, has made significant strides in space exploration, including successful satellite launches and missions like Chandrayaan and Mangalyaan, which have received international recognition.")

'positive'

In [30]:
classify_text("The central government has been making allegations of corruption in high-ranking positions.")

'negative'

In [31]:
classify_text("Farmers' protests intensify, putting pressure on the central government to reassess reforms.")

'negative'

In [32]:
classify_text("Sports Infrastructure Development: Investment in state-of-the-art sports facilities and stadiums for athletes.")

'neutral'

In [33]:
classify_text("விளையாட்டு உள்கட்டமைப்பு மேம்பாடு: விளையாட்டு வீரர்களுக்கான அதிநவீன விளையாட்டு வசதிகள் மற்றும் மைதானங்களில் முதலீடு.")

'positive'

In [34]:
classify_text("Gender Inequality: Lack of support and investment in women's sports, leading to disparities.")

'negative'

In [35]:
classify_text("பாலின சமத்துவமின்மை: பெண்களின் விளையாட்டுகளில் ஆதரவு மற்றும் முதலீடு இல்லாமை, ஏற்றத்தாழ்வுகளுக்கு வழிவகுக்கிறது.")

'negative'

In [36]:
classify_text("Violence in Sports: Incidents of violence or hooliganism at sporting events, raising concerns over security.")

'negative'

In [37]:
classify_text("விளையாட்டுகளில் வன்முறை: விளையாட்டு நிகழ்வுகளில் வன்முறை அல்லது போக்கிரித்தனமான சம்பவங்கள், பாதுகாப்பு குறித்த கவலைகளை எழுப்புகிறது.")

'negative'

In [38]:
classify_text("All eyes on KL Rahul. As he is coming back from injury")

'neutral'

In [39]:
classify_text("கேஎல் ராகுல் மீது அனைவரின் பார்வையும். அவர் காயத்தில் இருந்து மீண்டு வருவதால்")

'negative'

In [40]:
classify_text("Political Interference: Government interference in sports organizations' autonomy and decision-making.")

'negative'

In [41]:
classify_text("அரசியல் தலையீடு: விளையாட்டு நிறுவனங்களின் சுயாட்சி மற்றும் முடிவெடுப்பதில் அரசின் தலையீடு.")

'neutral'

In [42]:
# Save the fine-tuned model as a single file
name = "multilingual_new_classifier_xlm"
model.save_pretrained(name)
tokenizer.save_pretrained(name)


('multilingual_new_classifier_xlm/tokenizer_config.json',
 'multilingual_new_classifier_xlm/special_tokens_map.json',
 'multilingual_new_classifier_xlm/sentencepiece.bpe.model',
 'multilingual_new_classifier_xlm/added_tokens.json')

In [43]:
# torch.save(model.state_dict(), "mbert.pth")

In [46]:
!ls /content/multilingual_new_classifier_xlm -l

total 1091156
-rw-r--r-- 1 root root        924 Sep 22 17:52 config.json
-rw-r--r-- 1 root root 1112252785 Sep 22 17:52 pytorch_model.bin
-rw-r--r-- 1 root root    5069051 Sep 22 17:52 sentencepiece.bpe.model
-rw-r--r-- 1 root root        279 Sep 22 17:52 special_tokens_map.json
-rw-r--r-- 1 root root        443 Sep 22 17:52 tokenizer_config.json


In [44]:
# from google.colab import drive

# # Mount your Google Drive
# drive.mount('/content/drive')

# Specify the path to your file in Colab
colab_file_path = f'/content/{name}'

# Specify the destination path in your Google Drive
drive_file_path ='/content/drive/MyDrive'


# Copy the file from Colab to Google Drive
!cp "$colab_file_path" "$drive_file_path" -r


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [45]:
from google.colab import drive

# Mount your Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
